In [10]:
import pandas as pd
import numpy as np
import altair as alt

In [20]:
with open("/data2/elilouis/sublimationofsnow/thermistor_harp/nighttime_out_the_door.log") as f:
    lines = f.readlines()

In [26]:
lines[:5]

['MOT049  13.38  6.468 55.322 112.587\n',
 'TS302 -14.841858 1022927 -13.850372 1071731 -13.727966 1076290 -13.716309 1079207 -16.07\n',
 'TS301 -14.670837 1033014 -13.933166 1078793 -13.812378 1082868 -13.747192 1081226 -16.49\n',
 'MOT049  13.25  6.465 56.470 112.747\n',
 'TS302 -14.831879 1023434 -13.867554 1070830 -13.706818 1077406 -13.664062 1081970 -16.18\n']

In [27]:
cleaned_lines[:5]

['TS302 -14.841858 1022927 -13.850372 1071731 -13.727966 1076290 -13.716309 1079207 -16.07\n',
 'TS301 -14.670837 1033014 -13.933166 1078793 -13.812378 1082868 -13.747192 1081226 -16.49\n',
 'TS302 -14.831879 1023434 -13.867554 1070830 -13.706818 1077406 -13.664062 1081970 -16.18\n',
 'TS301 -14.655731 1033789 -13.919556 1079513 -13.769531 1085148 -13.701752 1083646 -16.61\n',
 'TS302 -14.822266 1023923 -13.727325 1078207 -13.601318 1082985 -13.566559 1087142 -16.28\n']

In [25]:
cleaned_lines = [f for f in lines if f.startswith('TS302') or f.startswith('TS301')]
delimited_lines = [line.split() for line in cleaned_lines]
temp_only_lines = [[token for token in line if token.startswith('TS') or '.' in token] for line in delimited_lines]
df = pd.DataFrame(temp_only_lines).drop(columns=5)

In [28]:
low_df = df[df[0]=='TS302']
low_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(low_df))
low_df = low_df.rename(columns={1:5, 2:15, 3:25, 4:35})
low_df = low_df.drop(columns=0)
low_df['side'] = 'low'
low_df

/tmp/ipykernel_359559/2556209242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(low_df))


,5,15,25,35,time_seconds,side
0,-14.841858,-13.850372,-13.727966,-13.716309,0,low
2,-14.831879,-13.867554,-13.706818,-13.664062,5,low
4,-14.822266,-13.727325,-13.601318,-13.566559,10,low
6,-14.813110,-13.923798,-13.826599,-13.868622,15,low
8,-14.805206,-14.132446,-14.051392,-14.078094,20,low
...,...,...,...,...,...,...
520,-15.891785,-16.232421,-15.967895,-15.877502,1300,low
522,-15.897369,-15.889191,-15.573700,-15.449493,1305,low
524,-15.902313,-15.638733,-15.626892,-15.578461,1310,low
526,-15.907684,-15.933563,-15.655273,-15.572906,1315,low


In [29]:
high_df = df[df[0]=='TS301']
high_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(high_df))
high_df = high_df.rename(columns={1:10, 2:20, 3:30, 4:40})
high_df = high_df.drop(columns=0)
high_df['side'] = 'high'
high_df

/tmp/ipykernel_359559/40103918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(high_df))


,10,20,30,40,time_seconds,side
1,-14.670837,-13.933166,-13.812378,-13.747192,0,high
3,-14.655731,-13.919556,-13.769531,-13.701752,5,high
5,-14.653351,-13.831055,-13.690491,-13.623444,10,high
7,-14.652039,-13.964966,-13.892120,-13.873718,15,high
9,-14.672272,-14.187286,-14.128601,-14.109100,20,high
...,...,...,...,...,...,...
519,-16.485015,-16.056762,-15.927704,-15.850098,1295,high
521,-16.489013,-16.217254,-16.010681,-15.891479,1300,high
523,-16.497711,-16.016326,-15.706421,-15.475220,1305,high
525,-16.485717,-15.805664,-15.563263,-15.359222,1310,high


In [30]:
df = pd.concat([
    low_df.melt(id_vars=['time_seconds', 'side']),
    high_df.melt(id_vars=['time_seconds', 'side'])
]).sort_values(['time_seconds', 'variable'])

In [31]:
df

,time_seconds,side,variable,value
0,0,low,5,-14.841858
0,0,high,10,-14.670837
265,0,low,15,-13.850372
264,0,high,20,-13.933166
530,0,low,25,-13.727966
...,...,...,...,...
1055,1315,high,40,-15.594971
264,1320,low,5,-15.910339
529,1320,low,15,-15.872314
794,1320,low,25,-15.626892


In [43]:
alt.Chart(df).mark_line().encode(
    alt.X('time_seconds:Q'),
    alt.Y('value:Q', scale=alt.Scale(zero=False)),
    alt.Color('variable:O')
).properties(width=1000)

/home/elilouis/mambaforge/envs/arm/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [33]:
alt.Chart(df).mark_line(point=True).encode(
    alt.X('value:Q', sort='-y'),
    alt.Y('variable:Q'),
    alt.Facet('time_seconds:Q')
).properties(width=100)

alt.Chart(...)